In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
train_dir = r"C:\Users\HP\Downloads\FireDataset-V6-JPG-Reshaped224\train"
val_dir = r"C:\Users\HP\Downloads\FireDataset-V6-JPG-Reshaped224\val"
test_dir = r"C:\Users\HP\Downloads\FireDataset-V6-JPG-Reshaped224\test"


In [3]:
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_data = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_data = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)


Found 4014 images belonging to 2 classes.
Found 492 images belonging to 2 classes.
Found 524 images belonging to 2 classes.


In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
checkpoint = ModelCheckpoint("best_fire_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[checkpoint]
)


C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 705ms/step - accuracy: 0.8482 - loss: 0.3433
Epoch 1: val_accuracy improved from -inf to 0.95325, saving model to best_fire_model.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 105s 778ms/step - accuracy: 0.8486 - loss: 0.3424 - val_accuracy: 0.9533 - val_loss: 0.1317
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 687ms/step - accuracy: 0.9416 - loss: 0.1505
Epoch 2: val_accuracy improved from 0.95325 to 0.95732, saving model to best_fire_model.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 93s 735ms/step - accuracy: 0.9417 - loss: 0.1504 - val_accuracy: 0.9573 - val_loss: 0.1321
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 677ms/step - accuracy: 0.9510 - loss: 0.1320
Epoch 3: val_accuracy improved from 0.95732 to 0.96951, saving model to best_fire_model.h5


126/126 ━━━━━━━━━━━━━━━━━━━━ 91s 723ms/step - accuracy: 0.9510 - loss: 0.1320 - val_accuracy: 0.9695 - val_loss: 0.0918
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 660ms/step - accuracy: 0.9551 - loss: 0.1093
Epoch 4: val_accuracy did not improve from 0.96951
126/126 ━━━━━━━━━━━━━━━━━━━━ 89s 704ms/step - accuracy: 0.9552 - loss: 0.1093 - val_accuracy: 0.9654 - val_loss: 0.1181
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9551 - loss: 0.1010
Epoch 5: val_accuracy did not improve from 0.96951
126/126 ━━━━━━━━━━━━━━━━━━━━ 316s 3s/step - accuracy: 0.9552 - loss: 0.1010 - val_accuracy: 0.9614 - val_loss: 0.0994
Epoch 6/10
 40/126 ━━━━━━━━━━━━━━━━━━━━ 57s 663ms/step - accuracy: 0.9658 - loss: 0.1108

In [ ]:
model = tf.keras.models.load_model("best_fire_model.h5")

# Predict on test data
preds = model.predict(test_data)
y_pred = (preds > 0.5).astype(int).flatten()
y_true = test_data.classes

# Evaluation report
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))
